In [ ]:
import os

from veil.veil_client import VeilClient
from veil.constants import NetworkId, NETWORK_INFO

NETWORK_ID = NetworkId.KOVAN
PRIVATE_KEY = os.environ.get("PRIVATE_KEY")
INFURA_API_KEY = os.environ.get("INFURA_API_KEY")
WEB3_RPC_URL = "https://{}.infura.io/{}".format(NETWORK_ID.name.lower(),INFURA_API_KEY)

client = VeilClient(
    network_id=NETWORK_ID.value,
    web3_rpc_url=WEB3_RPC_URL,
    private_key=PRIVATE_KEY,
)

MY_ADDRESS = client.account_address

print(
    "NETWORK:",  NETWORK_ID,
    "\nRPC_URL", WEB3_RPC_URL,
    "\nADDRESS IN USE:", MY_ADDRESS,
    "\nETH BALANCE:", client.get_eth_balance()
)

print(client)

In [ ]:
client.get_eth_balance()

In [ ]:
markets = client.get_markets(
    channel="btc", 
    status="open",
    page=0
)
print(len(markets))
markets[0]

In [ ]:
print(set([x["type"] for x in markets]))
print(set([x["channel"] for x in markets]))

In [ ]:
client.get_market("btc-usd-2019-03-03-5x")